<a href="https://colab.research.google.com/github/sivasurasani/Accessebility-ChatBot/blob/vinay-AI-Model/Simple_Transformers_Data_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json

# Read the Excel file into a DataFrame
df = pd.read_excel('AccessibilityChatbotData.xlsx')

In [2]:
column_to_json_key_mapping = {
    'Description': 'context1',
    'Sub Headings Description': 'context2',
    'Questions': 'question',
    'Answers': 'text',
    'AnswerStart': 'answer_start'
}


In [3]:
df['CombinedContext'] = df['Description'].fillna('') + " " + df['Sub Headings Description'].fillna('')


In [4]:
print(df[['Description', 'Sub Headings Description', 'CombinedContext']].head())

                                         Description  \
0  The BBC Mobile Accessibility Guidelines are a ...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                            Sub Headings Description  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  Alternative delivery formats are essential for...   

                                     CombinedContext  
0  The BBC Mobile Accessibility Guidelines are a ...  
1                                                     
2                                                     
3                                                     
4   Alternative delivery formats are essential fo..

In [5]:
# Group by the 'CombinedContext' column
grouped = df.groupby('CombinedContext')

data_list = []

for combined_context, group in grouped:
    qas_list = []
    for index, row in group.iterrows():
        question = row['Questions']
        answer_text = row['Answers']
        answer_start = row['CombinedContext'].find(str(answer_text))

        qas_dict = {
            "id": str(index),
            "question": question,
            "is_impossible": False if pd.notna(answer_text) and answer_start != -1 else True, 
            "answers": [] if pd.isna(answer_text) or answer_start == -1 else [{"text": answer_text, "answer_start": answer_start}]
        }

        qas_list.append(qas_dict)

    context_dict = {
        "context": combined_context,
        "qas": qas_list
    }

    data_list.append(context_dict)


In [6]:
import json

# Save the data_list to a JSON file
with open('transformed_data.json', 'w') as json_file:
    json.dump(data_list, json_file)


In [9]:
import pandas as pd
import json

# Load the Excel data
df = pd.read_excel('AccessibilityChatbotData.xlsx')

# Forward fill for context to handle NaN values
df['Sub Headings Description'].fillna(method='ffill', inplace=True)
df['Questions'].fillna('', inplace=True)
df['Answers'].fillna('', inplace=True)

# Group by 'Sub Headings Description' to generate the JSON data structure
grouped_df = df.groupby('Sub Headings Description')

formatted_data_list = []

for context, group in grouped_df:
    qas_list = []
    for index, row in group.iterrows():
        question = row['Questions']
        answer_text = row['Answers']

        qas_dict = {
            "id": str(index),
            "question": question,
            "is_impossible": False if pd.notna(answer_text) and answer_start != -1 else True,
            "answers": [{"text": answer_text}] if answer_text.strip() else []
        }

        qas_list.append(qas_dict)

    context_dict = {
        "context": context if context.strip() != "" else "No context provided.",
        "qas": qas_list
    }
    formatted_data_list.append(context_dict)

# Save the data list to a JSON file
with open("output_json_file.json", 'w') as json_file:
    json.dump(formatted_data_list, json_file)
